In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-31 08:19:12.995485: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 08:19:13.018785: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


CudaSupportError: Error at driver init: 
Call to cuInit results in CUDA_ERROR_NO_DEVICE (100):

In [2]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train', 'transactions.parquet'),engine="parquet",  part_mem_fraction=0.1)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid', 'transactions.parquet'),engine="parquet", part_mem_fraction=0.1)

In [3]:
customer_features = ['customer_id', 'age', 'FN', 'Active', 'club_member_status','fashion_news_frequency']
article_features = ['article_id', 'product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']

In [4]:
articles = cudf.read_parquet(os.path.join(config.data_raw_dir,'articles.parquet'))[article_features]
customers = cudf.read_parquet(os.path.join(config.data_raw_dir,'customers_enc.parquet'))[customer_features]

In [5]:
articles = nvt.Dataset(articles)
customers = nvt.Dataset(customers)

In [6]:
train = Dataset.merge(train, articles, on="article_id", how="inner")
train = Dataset.merge(train, customers, on="customer_id", how="inner")
valid = Dataset.merge(valid, articles, on="article_id", how="inner")
valid = Dataset.merge(valid, customers, on="customer_id", how="inner")
del articles, customers

In [7]:
# ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id','week']

article_id = ["article_id"] >> Categorify() >> TagAsItemID()
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()


price = ["price"] >> FillMissing(fill_val=0) >> Normalize() >> TagAsItemFeatures() >> Rename(postfix='_norm')

time_features = ["t_dat"]
session_time = (
    time_features >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

day = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.day)
        >> nvt.ops.Rename(name = 'day')
        >> Categorify()
        >> TagAsUserFeatures())
week = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.weekday)
        >> nvt.ops.Rename(name = 'week')
        >> Categorify()
        >> TagAsUserFeatures())
month = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.month)
        >> nvt.ops.Rename(name = 'month')
        >> Categorify()
        >> TagAsUserFeatures())
year = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.year)
        >> nvt.ops.Rename(name = 'year')
        >> Categorify()
        >> TagAsUserFeatures())


In [8]:
age_boundaries = list(np.arange(0,100,1))
age =  ['age'] >>  FillMissing(0) >> Bucketize(age_boundaries) >> Categorify() >> TagAsUserFeatures() 
cat_customer_columns = (['FN', 'Active', 'club_member_status','fashion_news_frequency'] 
            >> Categorify() 
            >> TagAsUserFeatures()
            )


In [9]:
# article_features = ['article_id', 'product_code', 'prod_name', 'product_type_no',
#        'product_type_name', 'product_group_name', 'graphical_appearance_no',
#        'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
#        'perceived_colour_value_id', 'perceived_colour_value_name',
#        'perceived_colour_master_id', 'perceived_colour_master_name',
#        'department_no', 'department_name', 'index_code', 'index_name',
#        'index_group_no', 'index_group_name', 'section_no', 'section_name',
#        'garment_group_no', 'garment_group_name', 'detail_desc']

cat_article_columns = (article_features 
              >> Categorify()  
              >> TagAsItemFeatures()
              >> Rename(postfix='_cat'))

In [10]:
features = article_id + customer_id + price + day + week + month + year + cat_customer_columns + age + cat_article_columns
wf = nvt.Workflow(features)
train = wf.fit_transform(train)
valid = wf.transform(valid)
train.to_parquet(os.path.join(config.data_processed_dir,'train_processed.parquet'))
valid.to_parquet(os.path.join(config.data_processed_dir,'validation_processed.parquet'))

In [11]:
train = Dataset(os.path.join(config.data_processed_dir, 'train_processed.parquet'))
valid = Dataset(os.path.join(config.data_processed_dir, 'validation_processed.parquet'))
schema = train.schema
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

In [12]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,article_id,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.article_id.parquet,55241.0,512.0,0.0,55240.0,article_id
1,customer_id,"(Tags.USER, Tags.CATEGORICAL, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.customer_id.parquet,780124.0,512.0,0.0,780123.0,customer_id
2,price_norm,"(Tags.CONTINUOUS, Tags.ITEM)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,day,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.day.parquet,34.0,16.0,0.0,33.0,day
4,week,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.week.parquet,10.0,16.0,0.0,9.0,week
5,month,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.month.parquet,11.0,16.0,0.0,10.0,month
6,year,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.year.parquet,5.0,16.0,0.0,4.0,year
7,FN,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.FN.parquet,4.0,16.0,0.0,3.0,FN
8,Active,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.Active.parquet,4.0,16.0,0.0,3.0,Active
9,club_member_status,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.club_member_status.parquet,6.0,16.0,0.0,5.0,club_member_status


In [16]:
import tensorflow as tf

In [24]:
model = mm.DeepFMModel(schema, embedding_dim=64, deep_block=mm.MLPBlock([256, 64])) 
model.compile(optimizer='adam') 
model.fit(
    train, 
    validation_data=valid, 
    batch_size=4000, 
    epochs=100)

ValueError: No targets found in schema. Please tag your targets or provide them as branches.

In [19]:
user_schema = train.schema.select_by_tag(Tags.USER)
user_inputs = mm.InputBlockV2(user_schema)
query = mm.Encoder(user_inputs, mm.MLPBlock([128, 64]))

item_schema = train.schema.select_by_tag(Tags.ITEM)
item_inputs = mm.InputBlockV2(item_schema,)
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, 64]))

model = mm.TwoTowerModelV2(query, candidate)
opt = tf.keras.optimizers.Adagrad()
model.compile(
    optimizer=opt, 
    run_eagerly=False, 
    metrics=[mm.RecallAt(30), mm.NDCGAt(30)],)
model.fit(
    train, 
    validation_data=valid, 
    batch_size=4000, 
    epochs=100)

Epoch 1/100


2023-08-30 11:26:22.696813: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2058/2058 [==============================] - ETA: 0s - loss: 8.2939 - recall_at_30: 0.0075 - ndcg_at_30: 0.0023 - regularization_loss: 0.0000e+00 - loss_batch: 8.2939

2023-08-30 11:27:46.093032: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2058/2058 [==============================] - 96s 45ms/step - loss: 8.2939 - recall_at_30: 0.0075 - ndcg_at_30: 0.0023 - regularization_loss: 0.0000e+00 - loss_batch: 8.2938 - val_loss: 8.2058 - val_recall_at_30: 0.0155 - val_ndcg_at_30: 0.0051 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 6.8864
Epoch 2/100
2058/2058 [==============================] - 60s 29ms/step - loss: 8.2939 - recall_at_30: 0.0075 - ndcg_at_30: 0.0023 - regularization_loss: 0.0000e+00 - loss_batch: 8.2937 - val_loss: 8.2058 - val_recall_at_30: 0.0152 - val_ndcg_at_30: 0.0050 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 6.8869
Epoch 3/100
2058/2058 [==============================] - 58s 28ms/step - loss: 8.2939 - recall_at_30: 0.0075 - ndcg_at_30: 0.0023 - regularization_loss: 0.0000e+00 - loss_batch: 8.2937 - val_loss: 8.2059 - val_recall_at_30: 0.0149 - val_ndcg_at_30: 0.0049 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 6.8872
Epoch 4/100
 529/2058 [======>.......................] 

KeyboardInterrupt: 

In [13]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([512, 256, 128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [14]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(30), mm.NDCGAt(30)])
early_stopping = EarlyStopping(monitor='val_ndcg_at_30', patience=5, restore_best_weights=True)
ree = model.fit(train, validation_data=valid, batch_size=4000, epochs=200, callbacks=[early_stopping])

Epoch 1/200


2023-08-30 11:23:21.698649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-30 11:23:21.824828: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


 530/2058 [======>.......................] - ETA: 1:09 - loss: 7.7433 - recall_at_30: 0.0295 - ndcg_at_30: 0.0095 - regularization_loss: 0.0000e+00 - loss_batch: 7.7433

KeyboardInterrupt: 